In [1]:
from deepface import DeepFace
from datetime import datetime
import time
import numpy as np

import cv2
import multiprocessing as mp
from multiprocessing import Queue
from queue import Empty, Full

import utils
from utils import *

import importlib
importlib.reload(DeepFace)
importlib.reload(utils)

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
import credentials
from credentials import *

In [4]:
from utils import *
importlib.reload(utils)

<module 'utils' from 'C:\\Users\\Antony\\Documents\\uChicago\\Winter 2022\\Music Thesis\\utils.py'>

In [2]:
## Frontend: facial recognition video and analyses loops

# Initialize queues between workers. Max size of 1
video_to_data_q = Queue(1) # video -> data analysis
data_to_music_q = Queue(1) # data -> music generation
data_to_video_q = Queue(1) # data -> video
music_to_video_q_1 = Queue(1)# music generation -> video
music_to_video_q_2 = Queue(1) # Countdown timer


video_loop_mp = mp.Process(target=utils.video_loop, args = (video_to_data_q, data_to_video_q, music_to_video_q_1,
                                                           music_to_video_q_2))
data_loop_mp = mp.Process(target=utils.analysis_loop, args = (video_to_data_q, data_to_music_q, data_to_video_q))

video_loop_mp.start()
data_loop_mp.start()

In [23]:
# Emergency stop to the loops if they become runaway

video_loop_mp.terminate()
data_loop_mp.terminate()

In [3]:
creds() # Client credentials access. Sets SPOTIPY_CLIENT, SPOTIPY_CLIENT_SECRET, SPOTIPY_REDIRECT_URI env vars

scope = ['user-top-read', 'user-read-playback-state', 'user-modify-playback-state', 'user-read-private']

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope, show_dialog = True))

user_top_songs = sp.current_user_top_tracks()
user_top_artists = sp.current_user_top_artists()
    
available_genres = sp.recommendation_genre_seeds()

user_top_genres = []
genre_overlap = []

# Gets user's region so only available songs are played
# TODO: figure out how to do this through the request header
user_country = sp.current_user()['country']

# Gets user's top songs
# Also makes overlap between available genre seeds and user top genres
# Makes a few pretty expensive calls to figure out the genre of each song. Could be removed
user_top_song_uris = []
for track in user_top_songs['items']:
    song_uri = track['uri']
    user_top_song_uris.append(song_uri)
    for artist in track['artists']:
        artist_uri = artist['uri']
        genres = sp.artist(artist_uri)['genres']
        for genre in genres:
            genre = '-'.join(genre.split())
            if genre not in user_top_genres:
                user_top_genres.append(genre)
            if (genre in available_genres['genres']) and (genre not in genre_overlap):
                genre_overlap.append(genre)
    

# Gets user's top genres from top artists
# Also makes overlap between available genre seeds and user top genres
# for artist in user_top_artists['items']:
#     genres = artist['genres']
#     for genre in genres:
#         genre = '-'.join(genre.split())
#         if genre not in user_top_genres:
#             user_top_genres.append(genre)
#         if (genre in available_genres['genres']) and (genre not in genre_overlap):
#             genre_overlap.append(genre)

In [4]:
# emotion_labels = ['angry', 'disgusted', 'scared', 'happy', 'sad', 'surprised', 'neutral']

angry_params = {
    'min_acousticness': 0,
    'min_danceability': 0.2,
    'min_energy': 0.4,
    'min_instrumentalness': 0,
    'min_liveness': 0,
    'min_loudness': -60,
    'min_popularity': 0,
#     'min_speechiness': 0,
    'min_tempo': 80,
    'min_valence': 0,
    
    'max_acousticness': 0.8,
    'max_danceability': 0.65,
    'max_energy': 1,
    'max_instrumentalness': 1,
    'max_liveness': 0.4,
    'max_loudness': 0,
    'max_popularity': 100,
    'max_speechiness': 1,
#     'max_tempo': 120,
    'max_valence': 0.6,
    
    'target_acousticness': 0.3,
    'target_danceability': 0.4,
    'target_energy': 0.65,
    'target_instrumentalness': 0.5,
    'target_liveness': 0.5,
#     'target_loudness': -60,
#     'target_popularity': 0.5,
    'target_speechiness': 0.2,
    'target_tempo': 140,
    'target_valence': 0.2    
}

disgusted_params = {
    'min_acousticness': 0,
    'min_danceability': 0.2,
    'min_energy': 0.4,
    'min_instrumentalness': 0,
    'min_liveness': 0,
    'min_loudness': -60,
    'min_popularity': 0,
    'min_speechiness': 0,
    'min_tempo': 0,
    'min_valence': 0,
    
    'max_acousticness': 1,
    'max_danceability': 0.6,
    'max_energy': 0.7,
    'max_instrumentalness': 1,
    'max_liveness': 0.4,
    'max_loudness': 0,
#     'max_popularity': 100,
    'max_speechiness': 1,
    'max_tempo': 120,
    'max_valence': 0.5,
    
#     'target_acousticness': 0.5,
    'target_danceability': 0.4,
    'target_energy': 0.55,
    'target_instrumentalness': 0.5,
    'target_liveness': 0.5,
    'target_loudness': -60,
#     'target_popularity': 0.5,
    'target_speechiness': 0.5,
#     'target_tempo': 0.5,
    'target_valence': 0.2
}

scared_params = {
    'min_acousticness': 0,
    'min_danceability': 0,
    'min_energy': 0,
    'min_instrumentalness': 0,
    'min_liveness': 0,
    'min_loudness': -60,
    'min_popularity': 0,
    'min_speechiness': 0,
    'min_tempo': 0,
    'min_valence': 0,
    
    'max_acousticness': 1,
    'max_danceability': 0.4,
    'max_energy': 0.5,
    'max_instrumentalness': 1,
    'max_liveness': 0.4,
#     'max_loudness': 0,
#     'max_popularity': 100,
    'max_speechiness': 0.5,
    'max_tempo': 120,
    'max_valence': 0.6,
    
    'target_acousticness': 0.5,
    'target_danceability': 0.2,
    'target_energy': 0.3,
    'target_instrumentalness': 0.5,
    'target_liveness': 0.5,
    'target_loudness': -60,
#     'target_popularity': 0.5,
    'target_speechiness': 0.5,
#     'target_tempo': 0.5,
    'target_valence': 0.35    
}

happy_params = {
    'min_acousticness': 0,
    'min_danceability': 0.5,
    'min_energy': 0.5,
    'min_instrumentalness': 0,
    'min_liveness': 0,
    'min_loudness': -60,
#     'min_popularity': 0,
    'min_speechiness': 0,
    'min_tempo': 60,
    'min_valence': 0.4,
    
    'max_acousticness': 1,
    'max_danceability': 1,
    'max_energy': 1,
    'max_instrumentalness': 1,
    'max_liveness': 0.4,
    'max_loudness': 0,
    'max_popularity': 100,
    'max_speechiness': 1,
    'max_tempo': 120,
    'max_valence': 1,
    
#     'target_acousticness': 0.5,
    'target_danceability': 0.8,
    'target_energy': 0.8,
#     'target_instrumentalness': 0.5,
    'target_liveness': 0.3,
#     'target_loudness': -60,
#     'target_popularity': 0.5,
#     'target_speechiness': 0.5,
#     'target_tempo': 0.5,
    'target_valence': 0.8    
}

sad_params = {
    'min_acousticness': 0,
    'min_danceability': 0,
    'min_energy': 0,
    'min_instrumentalness': 0,
    'min_liveness': 0,
    'min_loudness': -60,
    'min_popularity': 0,
#     'min_speechiness': 0,
    'min_tempo': 0,
    'min_valence': 0,
    
    'max_acousticness': 1,
    'max_danceability': 0.4,
    'max_energy': 0.5,
    'max_instrumentalness': 1,
    'max_liveness': 0.4,
    'max_loudness': 0,
#     'max_popularity': 100,
    'max_speechiness': 1,
    'max_tempo': 120,
    'max_valence': 0.4,
    
    'target_acousticness': 0.6,
    'target_danceability': 0.2,
    'target_energy': 0.2,
    'target_instrumentalness': 0.8,
    'target_liveness': 0.3,
#     'target_loudness': -60,
#     'target_popularity': 0.5,
    'target_speechiness': 0.5,
    'target_tempo': 80,
    'target_valence': 0.2  
}

surprised_params = {
    'min_acousticness': 0.3,
    'min_danceability': 0.4,
    'min_energy': 0.3,
#     'min_instrumentalness': 0,
    'min_liveness': 0,
    'min_loudness': -60,
    'min_popularity': 0,
    'min_speechiness': 0,
    'min_tempo': 0,
    'min_valence': 0.2,
    
    'max_acousticness': 1,
    'max_danceability': 0.6,
    'max_energy': 0.8,
    'max_instrumentalness': 1,
    'max_liveness': 0.4,
    'max_loudness': 0,
    'max_popularity': 100,
    'max_speechiness': 1,
#     'max_tempo': 120,
    'max_valence': 1,
    
    'target_acousticness': 0.5,
    'target_danceability': 0.5,
    'target_energy': 0.5,
    'target_instrumentalness': 0.5,
    'target_liveness': 0.5,
    'target_loudness': -60,
#     'target_popularity': 0.5,
    'target_speechiness': 0.5,
    'target_tempo': 108,
    'target_valence': 0.5    
}

neutral_params = {
    'min_acousticness': 0,
    'min_danceability': 0,
    'min_energy': 0.3,
    'min_instrumentalness': 0,
    'min_liveness': 0,
    'min_loudness': -60,
    'min_popularity': 0,
    'min_speechiness': 0,
    'min_tempo': 0,
    'min_valence': 0.3,
    
    'max_acousticness': 0.8,
    'max_danceability': 0.5,
    'max_energy': 0.7,
    'max_instrumentalness': 1,
    'max_liveness': 0.2,
    'max_loudness': 0,
    'max_popularity': 100,
    'max_speechiness': 0.6,
    'max_tempo': 140,
    'max_valence': 0.7,
    
    'target_acousticness': 0.5,
    'target_danceability': 0.4,
    'target_energy': 0.5,
    'target_instrumentalness': 0.5,
    'target_liveness': 0.5,
    'target_loudness': -60,
#     'target_popularity': 0.5,
    'target_speechiness': 0.5,
#     'target_tempo': 0.5,
    'target_valence': 0.45    
}

# emotion_labels = ['angry', 'disgusted', 'scared', 'happy', 'sad', 'surprised', 'neutral']

emotion_params = {'Angry': angry_params, 'Disgusted': disgusted_params, 'Scared': scared_params, 'Happy': happy_params, \
                  'Sad': sad_params, 'Surprised': surprised_params, 'Neutral': neutral_params}

In [5]:
from utils import *
importlib.reload(utils)

<module 'utils' from 'C:\\Users\\Antony\\Documents\\uChicago\\Winter 2022\\Music Thesis\\utils.py'>

In [5]:
pause_time = 10 # seconds

emotions_loop_mp = mp.Process(target=utils.music_loop, args = (data_to_music_q, music_to_video_q_1, music_to_video_q_2, 
                                                               pause_time, emotion_params, genre_overlap, 
                                                               user_top_songs, user_top_artists, user_country, sp))
emotions_loop_mp.start()

In [21]:
# Emergency stop to the loop if it becomes runaway
emotions_loop_mp.terminate()

In [8]:
sp.devices()

{'devices': [{'id': '6b335eda636e807e25d167a25a7456d9edbcc436',
   'is_active': True,
   'is_private_session': False,
   'is_restricted': False,
   'name': 'SM-G781V',
   'type': 'Smartphone',
   'volume_percent': 100}]}

In [7]:
user_top_genres

['gauze-pop',
 'indietronica',
 'shiver-pop',
 'australian-indie',
 'modern-rock',
 'indie-pop',
 'vancouver-indie',
 'a-cappella',
 'modern-alternative-rock',
 'pop',
 'shimmer-pop',
 'broadway',
 'hollywood',
 'show-tunes',
 'vocal-jazz',
 'alternative-hip-hop',
 'classical',
 'late-romantic-era',
 'russian-romanticism',
 'orchestral-performance',
 'british-orchestra',
 'classical-performance',
 'orchestra',
 'folk',
 'indie-folk',
 'new-americana',
 'belgian-pop',
 'g-house',
 'modern-alternative-pop',
 'ambient-pop',
 'dream-pop',
 'el-paso-indie',
 'shoegaze']

In [26]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope, show_dialog = True, open_browser = True))
user_top_songs = sp.current_user_top_tracks()
user_top_artists = sp.current_user_top_artists()

In [28]:
recs = sp.recommendations(seed_genres = {'bossanova'})

In [33]:
recs['tracks'][8]

{'album': {'album_type': 'ALBUM',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5HlnN6xF2MD87KhGRmCRTd'},
    'href': 'https://api.spotify.com/v1/artists/5HlnN6xF2MD87KhGRmCRTd',
    'id': '5HlnN6xF2MD87KhGRmCRTd',
    'name': 'Vinícius de Moraes',
    'type': 'artist',
    'uri': 'spotify:artist:5HlnN6xF2MD87KhGRmCRTd'},
   {'external_urls': {'spotify': 'https://open.spotify.com/artist/2it164hfV4gOIjzfGz7Q05'},
    'href': 'https://api.spotify.com/v1/artists/2it164hfV4gOIjzfGz7Q05',
    'id': '2it164hfV4gOIjzfGz7Q05',
    'name': 'Marilia Medalha',
    'type': 'artist',
    'uri': 'spotify:artist:2it164hfV4gOIjzfGz7Q05'},
   {'external_urls': {'spotify': 'https://open.spotify.com/artist/7rPqFVgJYARiFsLDlN6W6y'},
    'href': 'https://api.spotify.com/v1/artists/7rPqFVgJYARiFsLDlN6W6y',
    'id': '7rPqFVgJYARiFsLDlN6W6y',
    'name': 'Toquinho',
    'type': 'artist',
    'uri': 'spotify:artist:7rPqFVgJYARiFsLDlN6W6y'}],
  'available_markets': ['AD',
   'AE'